TO CHECK LATER : https://github.com/uclnlp/emoji2vec 

LEADER BOARD : https://www.kaggle.com/c/arabic-sentiment-analysis-2021-kaust/leaderboard 

In [1]:
import pandas as pd
import torch
import json
import codecs
import torch

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
repo_path = '/content/gdrive/My\ Drive/Colab\ Notebooks/arabic_sentiment_'
%cd $repo_path

/content/gdrive/My Drive/Colab Notebooks/arabic_sentiment_


In [6]:
!echo "# sentiment_analysis_arabic" >> README.md
!git init
!git add README.md

Initialized empty Git repository in /content/gdrive/My Drive/Colab Notebooks/arabic_sentiment_/.git/


In [36]:
!git config --global user.email "ch_khaldi@esi.dz"
!git config --global user.name "hadjerkhd"
!git config --global user.password "hadjira02/"

In [ ]:
!git commit -m "first commit"

In [38]:
!git branch -M main

In [39]:
!git remote set-url origin https://github.com/Hadjerkhd/sentiment_analysis_arabic.git

In [40]:
!git config --get remote.origin.url

https://github.com/Hadjerkhd/sentiment_analysis_arabic.git


In [41]:
!git push origin main

fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!pip install GPUtil pytorch_pretrained_bert transformers 

## Train Bert

In [ ]:

from tqdm.notebook import tqdm

from transformers import (AutoConfig, AutoModel,AutoTokenizer, AutoModelForSequenceClassification )
from torch.utils.data import TensorDataset


In [ ]:
df = pd.read_csv("train_data_h&a_split.csv")
df.head(100)

,Tweet_id,text,sentiment,label,data_type
0,1242480612440559616,@SaudiHousingCC \nهل احتساب قيمة دعم القرض تكو...,Neutral,0,train
1,1242480612440559616,@SaudiHousingCC \nهل احتساب قيمة دعم القرض تكو...,Neutral,0,train
2,1242425389726793728,@SaudiHousing لم يتم ايداع الدعم السكني في حسا...,Neutral,0,train
3,1242425389726793728,@SaudiHousing لم يتم ايداع الدعم السكني في حسا...,Neutral,0,train
4,1230120274932256768,@ejar_sa السلام عليكم حبيت اسال هل يلزم المست...,Neutral,0,train
...,...,...,...,...,...
95,1227302304568745984,احس في معدتي بركان من البعبع حق شاورمر 💔,Negative,1,train
96,1237084658157879296,@ejar_sa احتاج تفعيل حسابي لأني احاول اسجل دخو...,Neutral,0,train
97,1237084658157879296,@ejar_sa احتاج تفعيل حسابي لأني احاول اسجل دخو...,Neutral,0,train
98,1229606518850539520,@sakani_housing قرض الترميم لماذا لايشمل المس...,Neutral,0,train


In [ ]:

df.groupby(['sentiment', 'label', 'data_type']).count()

Tweet_id   text
sentiment label data_type                 
Negative  1     train          5243   5243
                val             582    582
Neutral   0     train         24387  24387
                val            2710   2710
Positive  2     train          5784   5784
                val             643    643

In [ ]:
possible_labels = df.sentiment.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
label_dict

{'Negative': 1, 'Neutral': 0, 'Positive': 2}

In [ ]:
model_name = "marBERTweightedloss"

# asafaya/bert-base-arabic 0.70601
tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERT")
model = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=True,
                                                     return_dict= True)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at U

In [ ]:
len(list(tokenizer.get_vocab()))

100000

In [ ]:
(list(df[df.data_type=='val'].text.values))

['حسبي الله عليك يا اسوء شركة اتصالات...\nتشحن رصيدك وتكتشف بعد يومين انهم سحبوا الرصيد كله وانت ما أجريت ولا اتصال\n\n@stc\n@stc_ksa',
 '@majedhogail اللهم ارزقني في بيت لي ولاولادي...العمر٥٢..اللهم امين يارب',
 '@SaudiHousingCC                             مذا تعني هذي الرسالة لوتكرمتم\n\nعزيزى علي محمد عبدالله الشهري تم تعليق الشهادة لغرض الاسترداد',
 'أنا شخص يغرقّك بالفرص بس ما تدري متى تكون الأخيرة.',
 'اليوم علق النت علي وقطع وانا اختبر شكرًا حبايبي @stc 🤝.',
 '@stc اتمنى اذا منتو راضين تردون ماتعلقون ابوي بالخط',
 'شفاء 4 حالات من المصابين بفيروس كورونا المستجد، ليصبح عدد المتعافين 43 حالة. حمى الله الجميع من كل شر\n#كورونا',
 '@MCgovSA دواجن الأخوين تدخر بعض    منتجاتها لشهر رمضان خاصة الصدور هل مسموح لهم ذلك.',
 '@ejar_sa هل استطيع اضافة بنود تخص شركتي على العقد الخاص بكم لأنه لدي بنود اساسية ومهمه ،، هل ممكن اضع عقد خاص بي بموافقتكم اوكيف؟؟',
 'احس في معدتي بركان من البعبع حق شاورمر 💔',
 'وزارة الصحة الكندية تعلن عن اكتشاف حالة أخرى جديدة مصابة بفيروس #كورونا قادمة من #مصر\n\

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    list(df[df.data_type=='train'].text.values), 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    list(df[df.data_type=='val'].text.values), 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(35414, 3935)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')        
        #print(f'F-score: {f1_score_func(y_preds[y_preds==label],labels)}\n')

In [ ]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:

device = torch.device('cuda')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
from torch.nn import MSELoss, CrossEntropyLoss, BCEWithLogitsLoss

def l2_loss(parameters):
    return torch.sum(
        torch.tensor([
            torch.sum(p ** 2) / 2 for p in parameters if p.requires_grad
        ]))

In [ ]:
loss = CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(input.size())
target = torch.empty(3, dtype=torch.long).random_(5)
print(target.size())
output = loss(input, target)
output.backward()

torch.Size([3, 5])
torch.Size([3])


In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        logits = outputs["logits"]#[0]
        #logits =  torch.argmax(logits, dim=1)
        labels = batch[2]
        device = logits.get_device()
        l2 = l2_loss(model.parameters())

        if device >= 0:
            l2 = l2.to(device)
        
        loss = l2 * 5e-3

        class_weights = [1,2,2]
        class_weights = torch.FloatTensor(class_weights).to(device)
        loss_fct = CrossEntropyLoss( weight=class_weights )
        loss += loss_fct(logits, labels)
        #print(loss)
        #loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    #torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 389.3184101732908
Validation loss: 0.25339193702358337
F1 Score (Weighted): 0.9391057432525862



Epoch 2
Training loss: 389.0979402764861
Validation loss: 0.1393948179946283
F1 Score (Weighted): 0.9754536755102602



Epoch 3
Training loss: 388.9979743690396
Validation loss: 0.12399403461196512
F1 Score (Weighted): 0.980755375336468



In [ ]:
model.state_dict()

OrderedDict([('bert.embeddings.position_ids',
              tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
                        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
                        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
                        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
                        56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
                        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
                        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
                        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
                       112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
                       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
                       1

In [ ]:
#torch.save(model.state_dict(), f''+model_name+'_epoch_3.model')
dir = "trained/"
model_name = 'marBERTweightedloss'
model.save_pretrained(dir+model_name+'_epoch_3.model')

In [ ]:
model.from_pretrained(dir+model_name+'_epoch_3.model')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
preds = np.argmax(predictions, axis=1).tolist()
labels = true_vals

In [ ]:
label_dict_inverse = {v: k for k, v in label_dict.items()}

In [ ]:
with open('preds_'+model_name+'.txt', 'a') as f:
    for i in range(len(preds)): 
        f.write(""+str(i)+"\t"+str(label_dict_inverse[preds[i]])+"\n")
f.close()

In [ ]:
with open('gold_'+model_name+'.txt', 'a') as f:
    for i in range(len(labels)): 
        f.write(""+str(i)+"\t"+str(label_dict_inverse[labels[i]])+"\n")
f.close()

In [ ]:
!perl score-semeval2016-task4.pl gold_marBERTweightedloss.txt preds_marBERTweightedloss.txt

Format OK
preds_marBERTweightedloss.txt	96.71	


In [ ]:
!cat preds_marBERTweightedloss.txt.scored

	Positive: P=98.91, R=95.35, F1=97.10
	Negative: P=96.91, R=95.76, F1=96.33
	 Neutral: P=98.12, R=99.25, F1=98.68
	OVERALL SCORE : 96.71


Positive: P=95.72, R=96.25, F1=95.9

	Negative: P=92.35, R=96.15, F1=94.22
	 Neutral: P=98.78, R=97.83, F1=98.30
	OVERALL SCORE : 95.10

# Test Bert

In [ ]:
#generate prediction for the competition 
import pandas as pd
import torch
import json
import codecs
import torch
from tqdm.notebook import tqdm

from transformers import (AutoConfig, AutoModel,AutoTokenizer, AutoModelForSequenceClassification )
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 1
device = torch.device('cuda')


In [ ]:
dir = "trained/"

#model_name = "Ar-BERTweightedloss"
label_dict = {'Neutral': 0, 'Negative': 1, 'Positive': 2}  
tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERT")
model = AutoModelForSequenceClassification.from_pretrained(dir+model_name+'_epoch_3.model')

In [ ]:
df_t = pd.read_csv("test_data.csv")
model.to(device)
labels = []
for index, row in df_t.iterrows():
  inputs = tokenizer(row['Text'], return_tensors="pt", return_token_type_ids=True, max_length=512, truncation=True)
  ids = inputs['input_ids'].to(device, dtype = torch.long)
  mask = inputs['attention_mask'].to(device, dtype = torch.long)
  token_type_ids = inputs['token_type_ids'].to(device, dtype=torch.long)
  outputs = model(ids, mask, token_type_ids)
  outputs = outputs['logits'].tolist()[0]
  label = outputs.index(max(outputs)) - 1
  labels.append(label)
#data = pd.read_csv('Test.csv', delimiter=',')

In [ ]:
df_t = pd.read_csv("test_data.csv")

df_t["sentiment"] = labels
df_t = df_t.drop(labels=["Text"], axis=1)
df_t['sentiment'][df_t['sentiment']==-1]= "Neutral"
df_t['sentiment'][df_t['sentiment']==0]= "Negative"
df_t['sentiment'][df_t['sentiment']==1]= "Positive"

df_t['sentiment'][df_t['sentiment']=="Neutral"]= 0
df_t['sentiment'][df_t['sentiment']=="Positive"]= 1
df_t['sentiment'][df_t['sentiment']=="Negative"]= -1

df_t#.to_csv("test_data_preds.csv", index=False)

,Tweet_id,sentiment
0,1251608623798722560,0
1,1221535162539872256,0
2,1252203833385779200,0
3,1221884276636938240,0
4,1252686719461273602,0
...,...,...
19995,1223396512266424321,0
19996,1147774058311933952,1
19997,1242834758893293571,-1
19998,1223192853876215808,1


In [ ]:
df_t.to_csv("test_data_preds_"+model_name+".csv",index=False)